In [30]:
import pandas as pd
import ms3 
from  ms3.utils import *
import os 
from ms3 import Score
import matplotlib.pyplot as plt

### Convert to mscx if not done yet

In [37]:
path_midi = '/home/nulpe/Desktop/Tresillo/dataset/project_midi/NonTresilloSongs/4;4/'
target = '/home/nulpe/Desktop/Tresillo/dataset/project_mscx/NonTresilloSongs_4_4/'

dir_list = os.listdir(path_midi)

for el in dir_list:
    convert(path_midi+el, target+el[:-4]+'.mscx', MS='musescore3')

INFO     root -- utils.py (line 427) convert():
	Converted /home/nulpe/Desktop/Tresillo/dataset/project_midi/NonTresilloSongs/4;4/Crazy little thing called love-Queen.mid to /home/nulpe/Desktop/Tresillo/dataset/project_mscx/NonTresilloSongs_4_4/Crazy little thing called love-Queen.mscx
INFO     root -- utils.py (line 427) convert():
	Converted /home/nulpe/Desktop/Tresillo/dataset/project_midi/NonTresilloSongs/4;4/Come_As_You_Are-Nirvana.mid to /home/nulpe/Desktop/Tresillo/dataset/project_mscx/NonTresilloSongs_4_4/Come_As_You_Are-Nirvana.mscx
INFO     root -- utils.py (line 427) convert():
	Converted /home/nulpe/Desktop/Tresillo/dataset/project_midi/NonTresilloSongs/4;4/I Shot the Sheriff-Eric Clapton.mid to /home/nulpe/Desktop/Tresillo/dataset/project_mscx/NonTresilloSongs_4_4/I Shot the Sheriff-Eric Clapton.mscx
INFO     root -- utils.py (line 427) convert():
	Converted /home/nulpe/Desktop/Tresillo/dataset/project_midi/NonTresilloSongs/4;4/Hey Jude- Beatles.mid to /home/nulpe/Desktop/

### Mscx to rythm vectors including all instruments

In [39]:
def rythm_vectors(in_dir, out_dir):
    list_sheet_music = os.listdir(in_dir)

    for idx, el in enumerate(list_sheet_music):
        #Get notes with onsets
        s = Score(dir_sheet_music+el)
        df = s.mscx.notes

        # Define instruments 
        instrument_dict = {}
        for key in s.mscx.metadata['parts']:
            for staff in s.mscx.metadata['parts'][key].keys():
                instrument_dict[staff] = key


        #staff to instruments 
        df['instrument'] = [instrument_dict[el] if el in instrument_dict else 'na' for el in df.staff]


        # define quarter beat 
        df['quarter_beats'] = (df.mc_onset*16).astype('int32')


        #make rythm matrix & data frame
        df_histogram = df.groupby(['instrument','quarter_beats'])['mn'].agg(['count'])
        df_histogram = df_histogram.groupby(level=0).apply(lambda x: x.reset_index(level = 0).drop(['instrument'],axis=1).reindex(list(range(0,16)),fill_value=0).T)
        df_histogram = df_histogram.reset_index()

        df_histogram.insert(loc=0, column='song_artist', value=el[:-5])

        #concat to big rythm vector df
        if idx == 0: df_rythm_vectors = df_histogram

        df_rythm_vectors = pd.concat([df_rythm_vectors,df_histogram], axis=0)

    df_rythm_vectors.to_csv(out_dir, index = False)

    
dir_sheet_music = '/home/nulpe/Desktop/Tresillo/dataset/project_mscx/NonTresilloSongs_4_4/'
out_dir = '/home/nulpe/Desktop/Tresillo/dataset/rythm_vectors/rythm_vectors_no_tresillos_4_4.csv'
rythm_vectors(dir_sheet_music, out_dir)

In [40]:
df_rythm_vectors = pd.read_csv('/home/nulpe/Desktop/Tresillo/dataset/rythm_vectors/rythm_vectors_tresillos_billboard.csv')
df_rythm_vectors

,song_artist,instrument,level_1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"Eastside-benny blanco, Halsey, Khalid",Athmosphere,count,32,0,0,0,0,0,0,0,30,0,0,0,30,0,0,0
1,"Eastside-benny blanco, Halsey, Khalid",Bright Piano,count,64,0,0,0,0,0,0,0,32,0,0,0,60,0,0,0
2,"Eastside-benny blanco, Halsey, Khalid",Fingered Bass,count,16,0,0,16,16,0,0,0,15,0,0,15,15,0,7,0
3,"Eastside-benny blanco, Halsey, Khalid",Jazz Guitar,count,48,0,0,0,0,0,0,0,45,0,0,0,45,0,0,0
4,"Eastside-benny blanco, Halsey, Khalid",Nylon Gtr.,count,48,0,0,0,0,0,0,0,45,0,0,0,45,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,"Cheap Thrills(feat. Sean Paul)-Sia, Sean Paul","Syn.Strings 1, Cheap Thrills",count,36,0,0,0,0,0,0,0,18,0,0,0,0,0,0,0
84,"Cheap Thrills(feat. Sean Paul)-Sia, Sean Paul",Synth Voice,count,12,0,0,0,6,0,6,0,12,0,0,0,6,0,0,0
85,"Cheap Thrills(feat. Sean Paul)-Sia, Sean Paul",Tenor Sax,count,18,2,11,4,11,0,9,0,17,5,4,2,11,3,7,4
86,"Cheap Thrills(feat. Sean Paul)-Sia, Sean Paul",Voice Oohs,count,1,0,0,0,0,0,10,0,11,0,10,0,10,0,0,0


### collapse instruments, one vector per song

In [53]:
df_rythm_vectors = pd.read_csv('/home/nulpe/Desktop/Tresillo/dataset/rythm_vectors/rythm_vectors_tresillos_billboard.csv')
rythm_vector_collaped = df_rythm_vectors.groupby(['song_artist']).agg(['sum'])
rythm_vector_collaped.columns = rythm_vector_collaped.columns.droplevel(1)
rythm_vector_collaped = rythm_vector_collaped.drop(['instrument', 'level_1'],axis=1)
rythm_vector_collaped

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
song_artist,,,,,,,,,,,,,,,,
"Cheap Thrills(feat. Sean Paul)-Sia, Sean Paul",406,14,29,196,119,0,202,0,382,17,32,199,129,3,184,4
"Eastside-benny blanco, Halsey, Khalid",628,18,54,182,120,42,150,30,490,22,56,176,532,64,178,92
"Let Me Love You ft Justin Bieber-DJ Snake, Justin Bieber",394,12,318,36,311,12,260,7,287,8,265,20,318,23,304,30
New rules-Dua Lipa,405,47,115,211,140,33,116,25,224,35,175,277,189,35,235,27
"Rockabye-Clean Bandit, Sean Paul, Anne-Marie",238,4,68,98,57,11,98,48,200,44,50,117,69,30,137,27
Shape of you-Ed Sheran,400,6,57,5,83,36,200,31,136,11,53,11,313,23,20,2
Treat you better-Shawn Mendes,279,44,68,161,120,46,159,43,258,43,55,163,125,61,178,57
